In [ ]:
# Install necessary libraries
!pip install nltk

# Import necessary libraries
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import random
import json
import pickle
import numpy as np
import tensorflow as tf
import os
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD

# Manually specify the NLTK data directory
nltk_dir = "/content/nltk_data"
os.makedirs(nltk_dir, exist_ok=True)
nltk.data.path.append(nltk_dir)

# Download necessary NLTK resources with force=True
nltk.download('punkt', download_dir=nltk_dir, force=True)
nltk.download('wordnet', download_dir=nltk_dir, force=True)
nltk.download('stopwords', download_dir=nltk_dir, force=True)
# Download 'punkt_tab' data
nltk.download('punkt_tab', download_dir=nltk_dir, force=True) # This line downloads the required 'punkt_tab' data

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Check for GPU availability
if tf.config.list_physical_devices('GPU'):
    print("GPU is available and will be used for training!")
else:
    print("GPU is not available. Training will use CPU.")

# Load intents from intents.json
with open('intents.json') as file:
    intents = json.load(file)

# Preprocess data
words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', '/', '@']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)  # Tokenize the sentence
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# Remove stopwords and lemmatize words
stop_words = set(stopwords.words('english'))
words = [
    lemmatizer.lemmatize(word.lower())
    for word in words
    if word not in ignore_letters and word.lower() not in stop_words
]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

# Save words and classes for inference
with open('words.pkl', 'wb') as f:
    pickle.dump(words, f)
with open('classes.pkl', 'wb') as f:
    pickle.dump(classes, f)

# Prepare training data
training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

# Shuffle and convert training data to NumPy arrays
random.shuffle(training)
training = np.array(training, dtype=object)

train_x = list(training[:, 0])
train_y = list(training[:, 1])

# Build the model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile the model using SGD optimizer
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Train the model
print("Training the model...")
hist = model.fit(np.array(train_x), np.array(train_y), epochs=40, batch_size=5, verbose=1)

# Save the trained model
model.save('chatbot_model.keras', hist)
print("Model saved as 'chatbot_model.keras'")

[nltk_data] Downloading package punkt to /content/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /content/nltk_data...
[nltk_data] Downloading package stopwords to /content/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /content/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


GPU is available and will be used for training!


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Training the model...
Epoch 1/40


/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


177/177 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.0722 - loss: 3.4106
Epoch 2/40
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3186 - loss: 2.5453
Epoch 3/40
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6341 - loss: 1.5438
Epoch 4/40
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7497 - loss: 0.9743
Epoch 5/40
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8135 - loss: 0.6668
Epoch 6/40
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8609 - loss: 0.5442
Epoch 7/40
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8751 - loss: 0.4962
Epoch 8/40
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8832 - loss: 0.3980
Epoch 9/40
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8841 - loss: 0.3878
Epoch 10/40
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8887 - loss: 0.3793
Epoch 11/40
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9162 - loss: 0.3473
Epoch 12/40
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy

In [ ]:
# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Load necessary files
try:
    with open('intents.json') as file:
        intents = json.load(file)

    with open('words.pkl', 'rb') as file:
        words = pickle.load(file)

    with open('classes.pkl', 'rb') as file:
        classes = pickle.load(file)

    model = load_model('chatbot_model.keras')
except FileNotFoundError as e:
    print(f"Error: {e}")
    exit(1)

# Preprocess input
def clean_up_sentence(sentence):
    sentence = sentence.lower()
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

# Convert input to bag-of-words format
def bow(sentence, words):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
    return np.array(bag)

# Predict the class of the input
def predict_class(sentence):
    bow_vector = bow(sentence, words)
    try:
        res = model.predict(np.array([bow_vector]))[0]
    except Exception as e:
        print(f"Prediction error: {e}")
        return []
    ERROR_THRESHOLD = 0.1
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return [{"intent": classes[r[0]], "probability": str(r[1])} for r in results]

# Get response based on predicted class
def get_response(intents_list, intents_json):
    if not intents_list:
        return "I'm sorry, I couldn't understand. Please try again."
    tag = intents_list[0]['intent']
    for intent in intents_json['intents']:
        if intent['tag'] == tag:
            return random.choice(intent['responses'])
    return "I'm sorry, I couldn't find a response for that."

# Run the chatbot
def chatbot():
    print(""" Chatbot is ready! Type 'exit' to stop.
  Welcome to medical chatbot please tell me your symptoms """)
    while True:
        user_input = input("You: ").strip()
        if user_input.lower() == "exit":
            print("Goodbye!")
            break

        intents_prediction = predict_class(user_input)
        if intents_prediction:
            print(f"Predicted intents: {intents_prediction}")  # Debugging line
        response = get_response(intents_prediction, intents)
        print(f"Bot: {response}")

if __name__ == "__main__":
    chatbot()


 Chatbot is ready! Type 'exit' to stop.
  Welcome to medical chatbot please tell me your symptoms 
You: hi
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 237ms/step
Predicted intents: [{'intent': 'greetings', 'probability': '0.5011247'}, {'intent': 'medical_greeting', 'probability': '0.41329113'}]
Bot: Hello!
You: i have sour throat and fever with headache
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Predicted intents: [{'intent': 'symptom_analysis', 'probability': '0.8709054'}, {'intent': 'remedy_request', 'probability': '0.11413503'}]
Bot: It seems you may have a common cold or flu. For a headache, consider taking paracetamol, and for a sore throat, warm salt water gargles may help. Consult a doctor for a detailed check-up.
You: exit
Goodbye!
